In [4]:
import os
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

class CNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        # conv block 1: in 1×64×64 → out 6×60×60 → pool → 6×30×30
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.relu  = nn.ReLU()
        self.pool  = nn.MaxPool2d(2, 2)
        # conv block 2: in 6×30×30 → out 16×26×26 → pool → 16×13×13
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        # fc layers: flatten 16×13×13 → 2704 → 120 → 84 → num_classes
        self.fc1   = nn.Linear(16 * 13 * 13, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

def load_data(data_dir='train', img_size=(64, 64)):
    X, Y = [], []
    class_names = sorted(os.listdir(data_dir))
    for idx, cls in enumerate(class_names):
        cls_dir = os.path.join(data_dir, cls)
        if not os.path.isdir(cls_dir):
            continue
        print(f'Loading class {cls}...')
        for fname in os.listdir(cls_dir):
            path = os.path.join(cls_dir, fname)
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            img = cv2.resize(img, img_size) / 255.0
            X.append(img.astype(np.float32))
            Y.append(idx)
    # Convert to numpy arrays: X → (N,1,64,64), Y → (N,)
    X = np.expand_dims(np.array(X), 1)
    Y = np.array(Y)
    return X, Y, class_names

def train(model, train_loader, test_loader, optimizer, epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    train_losses, test_losses, test_accuracies = [], [], []
    best_accuracy = 0
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(epochs):
        # --- training ---
        model.train()
        total_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        scheduler.step()
        train_losses.append(total_loss / len(train_loader))

        # --- evaluation ---
        model.eval()
        total_test_loss, correct, total = 0, 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                total_test_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (preds == labels).sum().item()
        epoch_loss = total_test_loss / len(test_loader)
        epoch_acc  = correct / total
        test_losses.append(epoch_loss)
        test_accuracies.append(epoch_acc)

        # save best
        if epoch_acc > best_accuracy:
            best_accuracy   = epoch_acc
            best_model_wts  = copy.deepcopy(model.state_dict())
            best_epoch      = epoch + 1

        print(f"Epoch {epoch+1}/{epochs}  "
              f"Train Loss: {train_losses[-1]:.4f}  "
              f"Test Loss: {epoch_loss:.4f}  "
              f"Test Acc: {epoch_acc:.4f}")

    # load best weights
    model.load_state_dict(best_model_wts)
    print(f"Loaded best model from epoch {best_epoch} with acc {best_accuracy:.4f}")

    # plot losses & accuracy
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(range(1,epochs+1), train_losses, label='Train Loss')
    plt.plot(range(1,epochs+1), test_losses, label='Test Loss')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
    plt.subplot(1,2,2)
    plt.plot(range(1,epochs+1), test_accuracies, label='Test Acc')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
    plt.tight_layout(); plt.show()

    return train_losses, test_losses, test_accuracies, best_accuracy

In [5]:
X, y, class_names = load_data(data_dir='data', img_size=(64, 64))
if len(X) == 0:
    print("No data found in 'train/' directory.")
    exit()

# 2) build dataset & loaders
X_tensor = torch.from_numpy(X)       # (N,1,64,64)
y_tensor = torch.from_numpy(y).long()
dataset  = TensorDataset(X_tensor, y_tensor)
n_test   = int(0.2 * len(dataset))
n_train  = len(dataset) - n_test
train_ds, test_ds = random_split(dataset, [n_train, n_test])
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds, batch_size=32)

# 3) model & optimizer
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model     = CNNClassifier(num_classes=len(class_names)).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs    = 10

# 4) train & evaluate
train_losses, test_losses, test_accuracies, best_acc = train(
    model, train_loader, test_loader, optimizer, epochs
)

Loading class A...
Loading class B...
Loading class C...
Loading class D...
Loading class E...
Loading class F...
Loading class G...


KeyboardInterrupt: 

In [3]:
# 5) save results
# serialize state_dict into memory and write to disk in chunks with progress bar
import io, math
from tqdm import tqdm
state_dict = model.cpu().state_dict()
buf = io.BytesIO()
torch.save(state_dict, buf, _use_new_zipfile_serialization=False)
buf.seek(0)
total_bytes = buf.getbuffer().nbytes
chunk_size = 1024 * 1024  # 1MB chunks
num_chunks = math.ceil(total_bytes / chunk_size)
with open('asl_cnn_best.pth', 'wb') as f:
    for chunk in tqdm(iter(lambda: buf.read(chunk_size), b''), total=num_chunks,
                      unit='B', unit_scale=True, desc='Saving model'):
        f.write(chunk)
print('Model saved successfully')

np.save('class_names.npy', class_names)
print(f"Training complete. Best test accuracy: {best_acc:.4f}")

Saving model: 100%|██████████| 2.00/2.00 [00:00<00:00, 1.04kB/s]

Model saved successfully
Training complete. Best test accuracy: 0.9877
